<div style="font-size: 13px; line-height: 1.4; margin: 0; padding: 0;">
<h5 style="margin-bottom: 0.2em; font-size: 14px;">
<b>Note:</b> This notebook contains testing and evaluation of the <code>codellama:7b-python</code> model conducted on <b>July 7, 2025</b>, with the goal of refining the system prompt for <code>Llama3.2</code> in <code>07July.ipynb</code>.
</h5>
</div>


In [47]:
# Install required packages if not already installed
# !pip install transformers accelerate

# If you see CUDA DLL errors, install CPU-only torch and set device_map to "cpu"
# %pip install torch --index-url https://download.pytorch.org/whl/cpu
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.pipelines import pipeline
import subprocess

model_id = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cpu")
llama_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="cpu")

OSError: [WinError 127] The specified procedure could not be found. Error loading "c:\Users\PC\anaconda3\envs\llms\Lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.

In [ ]:
# system_message_codellama = (
#     "ALWAYS output a complete C++17 program with a main() function that prints the result to standard output. "
#     "Your output MUST be a single, self-contained C++ file that compiles and runs as-is. "
#     "Do not omit the main() function. "
#     "You are a high-performance Python-to-C++ reimplementation assistant for Windows. "
#     "Always include all necessary headers: #include <iostream>, #include <chrono>, #include <cstdint>, #include <limits>, #include <iomanip>, #include <vector>, #include <cmath>. "
#     "Avoid using or inventing types that do not exist in standard C++ (such as uint128_t, uint256_t). Only use uint64_t, int64_t, double, etc. "
#     "Do not include numeric literals with underscores (such as 10_000_000), but rather only use standard C++ number literals (such as 10000000). "
#     "Avoid using template syntax like vector<int64_t>::size in numeric_limits or anywhere else. "
#     "Do not define or overload operators for built-in types, such as operator% for uint64_t. "
#     "Avoid custom namespaces and templates unless absolutely required by C++17. "
#     "Avoid Python-specific keywords (such as 'yield'). Avoid invalid casts or pointer casts in arithmetic; only use static_cast for valid type conversions. "
#     "Avoid using user-defined literals, literal suffixes that are not standard C++, such as 100_000_000 or 10ms)."
#     "Avoid old-style base class initializers and unnamed initializers in constructors unless absolutely required by C++17."
#     "All variables, constants, functions must be declared before their first use. This includes any constants (e.g., J, K) which should be declared as const variables within an appropriate scope such as global, namespace or function scope."
#     "Avoid using custom types that are not explicitly defined in the code and avoid invalid identifiers outside of C++17 standards."
#     "Use double quotes for string literals instead of single quotes. Always use std:: for standard library functions and manipulators (e.g., std::fixed, std::setprecision)."
#     "For random number generation: only allow custom linear congruential generators if the Python code uses it; never include <random> or std::mt19937_64 unless required by python."
#     "Use simple classes with next() methods for generators instead of complex iterators. Use int64_t consistently when dealing with 2^32 and use uint64_t only if necessary, such as in min_val = -10."
#     "Avoid incrementing const references or using undefined functions or variables; match variable names exactly from function parameters to prevent confusion between different scopes of the code. "
#     "Prefer simple structures over complex templates when possible – try not to use them unless absolutely required by C++17 standards." 
#     "Ensure that algorithms are identical in both Python and generated C++ versions, written with simplicity for readability."
#     "Only include standard library functions; no custom undefined ones should be used. All code must compile successfully using g++ -std=c++17 without requiring any features or syntax beyond c++17 standards." 
#     "For custom numeric types (e.g., Int64), provide non-explicit constructors for implicit conversions from standard integer types like int64_t and uint64_t, such as Int64(uint64_t val)."
#     "If a custom class has private data members that need external access, offer public getter methods (e.g., long long getValue() const;) or public conversion operators (e.g., operator int64_t() const;)"
#     "For custom numeric types: explicitly overload all necessary arithmetic and comparison operators (e.g., +, -, *, /, %, ==, !=, <, <=, >, >=) to support interactions with both other custom types as well as built-in ones like int64_t or uint64_t. Ensure symmetric operations are supported too – e.g., CustomType op BuiltInType and BuiltInType op CustomType."
#     "For floating-point modulo operations: always use std::fmod from the <cmath> header instead of % operator which is strictly for integer types only." 
#     "Provide valid C++ code that compiles without errors or warnings. "
#     "Output a complete C++17 program with a main() function that prints the result."
# )

In [ ]:
def user_prompt_for(python):
    user_prompt = (
        "IMPORTANT: Output a complete, compilable C++17 program with a main() function that prints the result. "
        "Do not omit the main() function. "
        "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
        "Respond only with C++ code; do not explain your work other than a few comments. "
        "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    )
    user_prompt += python
    return user_prompt

In [ ]:
def stream_llama_hf(python_code, max_new_tokens=2048):
    system_prompt = (
        "IMPORTANT: Output a complete, compilable C++17 program with a main() function that prints the result. "
        "Do not omit the main() function. "
        "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
        "Respond only with C++ code; do not explain your work other than a few comments. "
        "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    )
    prompt = system_prompt + python_code
    # Generate C++ code
    output = llama_pipe(prompt, max_new_tokens=max_new_tokens, do_sample=True)
    # Yield the generated code (simulate streaming)
    for line in output[0]['generated_text'].splitlines():
        yield line + "\n"

def write_output(cpp, filename="optimized_codellama.cpp"):
    code = cpp.replace("```cpp","").replace("```","")
    lines = code.split('\n')
    cleaned_lines = []
    for line in lines:
        stripped = line.strip()
        if (not stripped.startswith('Note that') and 
            not stripped.startswith('The ') and
            not stripped.startswith('This ') and
            '`' not in stripped): 
            cleaned_lines.append(line)
    cleaned_code = '\n'.join(cleaned_lines)
    with open(filename, "w") as f:
        f.write(cleaned_code)

In [ ]:
def execute_cpp(code):
    write_output(code, "optimized_codellama.cpp")
    try:
        compile_cmd = ["g++", "-O3", "-std=c++17", "-o", "optimized_codellama.exe", "optimized_codellama.cpp"]
        compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
        run_cmd = ["optimized_codellama.exe"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [ ]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
def run_llama_hf_on_problem(python_code, output_cpp="optimized_llama31.cpp"):
    print("=== Generating with Llama-3.1-8B-Instruct ===")
    cpp_code = ""
    for chunk in stream_llama_hf(python_code):
        print(chunk, end='', flush=True)
        cpp_code += chunk
    print("\n=== Writing and compiling ===")
    write_output(cpp_code, output_cpp)
    compile_cmd = ["g++", "-O3", "-std=c++17", "-o", "optimized_llama31.exe", output_cpp]
    compile_result = subprocess.run(compile_cmd, capture_output=True, text=True)
    if compile_result.returncode != 0:
        print(f"Compilation error: {compile_result.stderr}")
        return
    print("=== Running executable ===")
    run_cmd = ["optimized_llama31.exe"]
    run_result = subprocess.run(run_cmd, capture_output=True, text=True)
    print(run_result.stdout)

In [ ]:
# --- Replace your test cells with these ---
print("Testing Pi Calculation with Llama-3.1-8B-Instruct")
run_llama_hf_on_problem(pi)

print("Testing Maximum Subarray Sum with Llama-3.1-8B-Instruct")
run_llama_hf_on_problem(python_hard)